In [1]:
from random import sample
import serial
import serial.tools.list_ports
import time
import serial.tools.list_ports
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import numpy as np
import tensorflow as tf
from tensorflow import keras
ports = serial.tools.list_ports.comports()

In [2]:
def sensor_read(ax, ay, az, gx, gy, gz):    
    try:
        b = ser.readline()
        data_ser = b.decode().splitlines()
        ax_ser, ay_ser, az_ser, gx_ser, gy_ser, gz_ser = data_ser[0].split(",")
        ax.append(ax_ser)
        ay.append(ay_ser)
        az.append(az_ser)
        gx.append(gx_ser)
        gy.append(gy_ser)
        gz.append(gz_ser)
    except Exception as e:
        print(e)
    

In [3]:
def process_input_data(input:pd.DataFrame, ax_popped:list, ay_popped:list, az_popped:list, gx_popped:list, gy_popped:list, gz_popped:list):
    ax_popped = pd.DataFrame(ax_popped)
    ax_mean = ax_popped.abs().mean()
    ax_std = ax_popped.std()

    ay_popped = pd.DataFrame(ay_popped)
    ay_mean = ay_popped.abs().mean()
    ay_std = ay_popped.std()

    az_popped = pd.DataFrame(az_popped)
    az_mean = az_popped.abs().mean()
    az_std = az_popped.std()

    gx_popped = pd.DataFrame(gx_popped)
    gx_mean = gx_popped.abs().mean()
    gx_std = gx_popped.std()

    gy_popped = pd.DataFrame(gy_popped)
    gy_mean = gy_popped.abs().mean()
    gy_std = gy_popped.std()

    gz_popped = pd.DataFrame(gz_popped)
    gz_mean = gz_popped.abs().mean()
    gz_std = gz_popped.std()

    input['a_mean'] = pow(pow(ax_mean, 2) + pow(ay_mean, 2) + pow(az_mean, 2), 1/2)
    input['g_mean'] = pow(pow(gx_mean, 2) + pow(gy_mean, 2) + pow(gz_mean, 2), 1/2)
    input['a_std'] = pow(pow(ax_std, 2) + pow(ay_std, 2) + pow(az_std, 2), 1/2)
    input['g_std'] = pow(pow(gx_std, 2) + pow(gy_std, 2) + pow(gz_std, 2), 1/2)



In [4]:
def process_input_data(input_model_buffer:pd.DataFrame, ax_popped:list, ay_popped:list, az_popped:list, gx_popped:list, gy_popped:list, gz_popped:list):
    ax_popped = pd.DataFrame(ax_popped)
    ax_mean = ax_popped.abs().mean()
    ax_std = ax_popped.std()

    ay_popped = pd.DataFrame(ay_popped)
    ay_mean = ay_popped.abs().mean()
    ay_std = ay_popped.std()

    az_popped = pd.DataFrame(az_popped)
    az_mean = az_popped.abs().mean()
    az_std = az_popped.std()

    gx_popped = pd.DataFrame(gx_popped)
    gx_mean = gx_popped.abs().mean()
    gx_std = gx_popped.std()

    gy_popped = pd.DataFrame(gy_popped)
    gy_mean = gy_popped.abs().mean()
    gy_std = gy_popped.std()

    gz_popped = pd.DataFrame(gz_popped)
    gz_mean = gz_popped.abs().mean()
    gz_std = gz_popped.std()

    input_model_buffer['a_mean'] = pow(pow(ax_mean, 2) + pow(ay_mean, 2) + pow(az_mean, 2), 1/2)
    input_model_buffer['g_mean'] = pow(pow(gx_mean, 2) + pow(gy_mean, 2) + pow(gz_mean, 2), 1/2)
    input_model_buffer['a_std'] = pow(pow(ax_std, 2) + pow(ay_std, 2) + pow(az_std, 2), 1/2)
    input_model_buffer['g_std'] = pow(pow(gx_std, 2) + pow(gy_std, 2) + pow(gz_std, 2), 1/2)

In [5]:
reloaded_model = tf.keras.models.load_model('D:\laragon\www\\boxqt\CNN\model')

In [8]:
def Dequeue(window: list, pop_length: int):
    for i in range(len(window)):
        for j in range(pop_length):
          window[i].pop(0)

In [ ]:
# POP DATA FROM PIPE AND STORE TO WINDOW
def Enqueue(from_pipe:list, to_window:list, pop_length:int):
    for i in range(pop_length):
        for j in range(6):
            to_window[j].append(float(from_pipe[j].pop(0)))

In [10]:
input_buffer = []

ax = []
ay = []
az = []
gx = []
gy = []
gz = []
pipe = [ax, ay, az, gx, gy, gz]

ax_window = []
ay_window = []
az_window = []
gx_window = []
gy_window = []
gz_window = []
window = [ax_window, ay_window, az_window, gx_window, gy_window, gz_window]

start = time.time()
end = time.time()
i = 0
ser = serial.Serial(port="COM6", baudrate=1000 * 1000)
while end - start < 2:
    end = time.time()
    # GET DATA FROM SENSOR & STORE TO PIPE 
    # TODO: CONDITION GET DATA EVERY 1ms
    sensor_read(pipe[0], pipe[1], pipe[2], pipe[3], pipe[4], pipe[5])
    
Enqueue(pipe, window, 20)

# data_pop(pipe, window, 10)

def show_data_list(list:list):
    for i in range(len(list)):
        print(list[i])
        print(len(list[i]))

# show_data_list(window)
    
input = pd.DataFrame()
process_input_data(input, window[0], window[1], window[2], window[3], window[4], window[5])
test_predictions = reloaded_model.predict(input).flatten()

print("Process time: " + str(end - start))
print(input)
print(test_predictions)

ser.close()

'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte
1/1 [==============================] - 0s 18ms/step
Process time: 2.00064754486084
      a_mean    g_mean     a_std     g_std
0  11.563887  2.608861  5.439766  3.321819
[2786.1228]


In [9]:
ser.close()


In [11]:
output = []
for i in range(100):
     Dequeue(window, 10)
     Enqueue(pipe, window, 10)
     process_input_data(input, window[0], window[1], window[2], window[3], window[4], window[5])
     test_predictions = reloaded_model.predict(input).flatten()
     output.append(test_predictions)


1/1 [==============================] - 0s 18ms/step
